In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

┌ Info: Loading DataFrames support into Gadfly.jl
└ @ Gadfly /Users/rob/.julia/packages/Gadfly/09PWZ/src/mapping.jl:228


Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)

# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:31:01 of 0:31:03 remaining]
Chain 1:  10% [0:01:04 of 0:01:11 remaining]
Chain 1:  20% [0:00:55 of 0:01:09 remaining]
Chain 1:  30% [0:00:48 of 0:01:08 remaining]
Chain 1:  40% [0:00:41 of 0:01:08 remaining]
Chain 1:  50% [0:00:36 of 0:01:11 remaining]
Chain 1:  60% [0:00:29 of 0:01:12 remaining]
Chain 1:  70% [0:00:22 of 0:01:12 remaining]
Chain 1:  80% [0:00:14 of 0:01:12 remaining]
Chain 1:  90% [0:00:07 of 0:01:12 remaining]
Chain 1: 100% [0:00:00 of 0:01:12 remaining]

Chain 2:   0% [0:00:02 of 0:00:02 remaining]
Chain 2:  10% [0:00:45 of 0:00:50 remaining]
Chain 2:  20% [0:00:45 of 0:00:56 remaining]
Chain 2:  30% [0:00:42 of 0:01:00 remaining]
Chain 2:  40% [0:00:38 of 0:01:03 remaining]
Chain 2:  50% [0:00:32 of 0:01:04 remaining]
Chain 2:  60% [0:00:26 of 0:01:05 remaining]
Chain 2:  70% [0:00:20 of 0:01:05 remaining]
Chain 2:  80% [0:00:13 of 0:01:05 remaining]
Chain 2:  90% [0:00:07 of 0:01:05 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[25.1961 113.071 0.919993; 24.8087 113.071 0.926044; … ; 29.8158 114.485 0.890426; 26.0051 113.899 0.901715]

[25.7523 117.717 0.834315; 25.6858 117.717 0.834315; … ; 26.0071 111.551 0.94341; 27.0324 111.544 0.967398]

[27.0487 114.382 0.899885; 27.4818 115.444 0.876222; … ; 26.6821 116.132 0.858241; 27.3363 116.132 0.858241]

Show draws summary

In [3]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean         SD        Naive SE       MCSE        ESS   
     s2  26.1441091 1.980847505 0.01205505402 0.0182478740 9000.0000
beta[1] 113.9459360 1.939962118 0.01180623348 0.0366034012 2808.9474
beta[2]   0.9035289 0.042701954 0.00025987582 0.0007981684 2862.2423

Quantiles:
            2.5%        25.0%       50.0%       75.0%        97.5%   
     s2  22.51738946  24.7712309  26.0546627  27.4227062  30.27164315
beta[1] 110.12664291 112.6598691 113.9078436 115.2530528 117.79406309
beta[2]   0.81877964   0.8752713   0.9042416   0.9319756   0.98766346



End of `clip_38.0m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*